In [1]:
from Idntimes import Idntimes
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
obj = Idntimes()

In [2]:
end_date = date(2018, 8, 21)
start_date = date(2018, 8, 19)
init_page = 3

In [3]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], 'news', init_page, single_date.strftime("%Y-%m-%d"))
    init_page = 1

2018-08-20
page  1
https://www.idntimes.com/ajax/index?category=news&type=all&page=1&date=2018-08-20
https://www.idntimes.com/news/world/dian-f-hanum/gadis-8-tahun-sumbang-celengan-korban-banjir-c1c2
memasukkan berita id  1534698098
Insert berita  Mengharukan! Gadis 8 Tahun Ini Sumbang Celengannya untuk Korban Banjir
masuk
https://www.idntimes.com/news/indonesia/ita-malau/kemenag-jemaah-haji-jangan-panik-saat-ada-badai
memasukkan berita id  1534698097
Insert berita  Kemenag: Jemaah Haji Jangan Panik Saat Ada Badai
masuk
https://www.idntimes.com/news/world/dian-f-hanum/bawa-pisau-nenek-di-stun-gun-polisi-c1c2
memasukkan berita id  1534698089
Insert berita  Bawa Pisau Buat Memotong Bunga, Nenek 87 Tahun Di-Stun-Gun Polisi
masuk
https://www.idntimes.com/news/indonesia/indianamalia/jalani-kemo-kanker-sutopo-bnpb-tetap-semangat-infokan-bencana
memasukkan berita id  1534698114
Insert berita  Jalani Kemo Kanker, Sutopo BNPB Tetap Semangat Infokan Bencana
masuk
https://www.idntimes.com/news/wo

masuk
https://www.idntimes.com/news/indonesia/teatrika/gerindra-sandiaga-bisa-menarik-suara-pemilih-millennials
memasukkan berita id  1534698105
Insert berita  Gerindra: Sandiaga Bisa Menarik Suara Pemilih Millennials
masuk
https://www.idntimes.com/news/indonesia/ardiansyah-fajar/pesta-miras-di-gresik-korban-diduga-mencampurnya-dengan-spirtus
memasukkan berita id  1534698120
Insert berita  Pesta Miras di Gresik, Korban Diduga Mencampurnya dengan Spirtus
masuk
https://www.idntimes.com/news/indonesia/afrianisusanti/andi-arif-dilaporkan-sebagai-saksi-kasus-mahar-politik
memasukkan berita id  1534698109
Insert berita  Andi Arif Dilaporkan Sebagai Saksi Kasus Mahar Politik
masuk
https://www.idntimes.com/news/indonesia/ardiansyah-fajar/pemuda-tewas-akibat-miras-alami-kerusakan-seluruh-organ
memasukkan berita id  1534698112
Insert berita  Pemuda Tewas Akibat Miras Alami Kerusakan Seluruh Organ
masuk
https://www.idntimes.com/news/indonesia/rochmanudin-wijaya/gempa-susulan-di-lombok-101-kali-ad